In [40]:
import re, string
import nltk
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor,LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta

from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV 

from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer     
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers.core import Dropout
from keras.layers import LSTM
from keras.layers import Dense, Input, Flatten 
from keras.layers import Conv1D, MaxPooling1D, Embedding 
from keras.models import Model 
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from keras.models import load_model
from keras.applications import imagenet_utils
from keras.layers import Embedding
from keras.utils.np_utils import to_categorical
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.layers import initializers 
from keras.layers import regularizers 
from keras.layers import constraints 
from keras.layers import Activation
from keras.layers.advanced_activations import PReLU
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ELU
from keras.constraints import max_norm
import keras.backend as K

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')

In [2]:
coffee = pd.read_csv('./updated_coffee_df.csv')

In [3]:
coffee['Date'] = pd.to_datetime(coffee['Date'])
coffee.head()

,Unnamed: 0,Unnamed: 0.1,year,month,day,level_3,Title,Date,Price,Price_Change,Direction,Rate_of_Change
0,0,0,2007,1,2,Title,India earns more from higher coffee exports in...,2007-01-02,1.1506,0.0000,0,0.000000
1,1,1,2007,1,3,Title,Friesland raises stake in Indonesian subsidiar...,2007-01-03,1.1760,0.0254,0,0.022075
2,2,2,2007,1,4,Title,Nymex announces start date for soft commodity ...,2007-01-04,1.1451,-0.0309,0,-0.026276
3,3,3,2007,1,5,Title,India's largest coffee chain extends to Pakistan,2007-01-05,1.1506,0.0055,0,0.004803
4,4,4,2007,1,7,Title,Honduran coffee sales Ugandan coffee funds Soy...,2007-01-07,1.1506,-0.0000,0,0.000000


In [4]:
X = coffee[['Title', 'Date', 'Price_Change', 'Rate_of_Change']]
y = coffee['Price']

In [5]:
thedata = coffee
thedata['Title'] = thedata['Title'].str.encode('utf-8')
thedata['Title'] = thedata['Title'].str.lower()

In [6]:
##########################################
# clean up the text in the data with regex
##########################################
def clean_text(row):
    text = str(row['Title'])

    # Remove newline characters
    cleantext = text.replace('\r\n', ' ')

    # Convert HTML punctuation chaaracters
    cleantext = cleantext.replace(' www.', ' ')   
    cleantext = cleantext.replace('.com ', ' ')    
    cleantext = cleantext.replace('.', ' ')
    cleantext = cleantext.replace(',', ' ')
    cleantext = cleantext.replace('!', ' ')
    cleantext = cleantext.replace('$;', ' ')
    cleantext = cleantext.replace(';', ' ')
    cleantext = cleantext.replace(')', ' ')
    cleantext = cleantext.replace('(', ' ')   
    cleantext = cleantext.replace('>', ' ')  
    cleantext = cleantext.replace('<', ' ')  
    cleantext = cleantext.replace('-', ' ')  #take away hyphen and collapse hyphenated words
    cleantext = cleantext.replace(' the ', ' ')  
    cleantext = cleantext.replace(' of ', ' ')   
    cleantext = cleantext.replace(' in ', ' ')  
    cleantext = cleantext.replace(' and ', ' ')  
    cleantext = cleantext.replace(' by ', ' ')  
    cleantext = cleantext.replace(' to ', ' ')  
    cleantext = cleantext.replace(' at ', ' ')  
    cleantext = cleantext.replace(' on ', ' ') 
    cleantext = cleantext.replace(' for ', ' ')  
#    cleantext = cleantext.replace(' be ', ' ')   
    cleantext = cleantext.replace(' is ', ' ')    
    cleantext = cleantext.replace(' or ', ' ')   
#    cleantext = cleantext.replace(' we ', ' ')   
#    cleantext = cleantext.replace(' that ', ' ')   
#    cleantext = cleantext.replace(' our ', ' ')   
    cleantext = cleantext.replace(' as ', ' ')        
    cleantext = cleantext.replace(' from ', ' ')   
#    cleantext = cleantext.replace(' are ', ' ')   
    cleantext = cleantext.replace(' with ', ' ')   
#    cleantext = cleantext.replace(' us ', ' ')   
#    cleantext = cleantext.replace(' was ', ' ')        
    cleantext = cleantext.replace(' this ', ' ')   
    cleantext = cleantext.replace(' an ', ' ')        
    cleantext = cleantext.replace(' by ', ' ')   
    cleantext = cleantext.replace(' sr ', ' ')      
    cleantext = cleantext.replace(' it ', ' ')  
    cleantext = cleantext.replace(' s ', ' ')   


    #remove non alpha characters and specific noise
    cleantext = re.sub(r'\d+', ' ',cleantext)
    cleantext = re.sub(r'^b',' ',cleantext)
    cleantext = re.sub(r'[^\w]',' ',cleantext)
    cleantext = cleantext.replace('xc xs', ' ')  
    cleantext = cleantext.replace('xe xs', ' ')  
    cleantext = cleantext.replace('xc xS', ' ')  
    cleantext = cleantext.replace('xe xS', ' ')  
    cleantext = cleantext.replace('xc xa', ' ')  
    cleantext = cleantext.replace('xe xa', ' ')  
    cleantext = cleantext.replace(' xc xc x', ' ')  
    cleantext = cleantext.replace(' xc ', ' ')  
    cleantext = cleantext.replace(' xe ', ' ')  
    cleantext = cleantext.replace(' xs ', ' ')  
    cleantext = cleantext.replace(' xa ', ' ')  
    cleantext = cleantext.replace(' ct ', ' ')  
    cleantext = cleantext.replace(' x ', ' ')  
    cleantext = cleantext.replace(' non exclusive ', ' non-exclusive ') 
    cleantext = cleantext.replace(' u ', ' ')   
    cleantext = cleantext.replace(' s ', ' ')  
    
    #remove specific noise
    cleantext = cleantext.translate(str.maketrans({'‘':' ','’':' '}))
    cleantext = cleantext.translate(str.maketrans({',':' ',',':' '}))
    cleantext = cleantext.translate(str.maketrans({'[':' ',']':' '}))
    cleantext = cleantext.translate(str.maketrans({'"':' ','%':' '}))
    cleantext = cleantext.translate(str.maketrans({'^':' ','*':' '}))

    #remove punctuation
    punctpattern = re.compile('[%s]' % re.escape(string.punctuation))
    cleanttext = re.sub(punctpattern,'', cleantext)

    #remove single letter word
    cleantext = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', cleantext) 

    # Remove extra spaces
    cleantext = re.sub('\s+', ' ', cleantext).strip()

    return cleantext


In [7]:
#apply text fixes to the input text column
thedata['Clean_Text'] = thedata.apply(clean_text, axis=1)
justcleandocs=thedata.drop(['Title'], axis=1)


In [8]:
justcleandocs.head()


,Unnamed: 0,Unnamed: 0.1,year,month,day,level_3,Date,Price,Price_Change,Direction,Rate_of_Change,Clean_Text
0,0,0,2007,1,2,Title,2007-01-02,1.1506,0.0000,0,0.000000,india earns more higher coffee exports
1,1,1,2007,1,3,Title,2007-01-03,1.1760,0.0254,0,0.022075,friesland raises stake indonesian subsidiary m...
2,2,2,2007,1,4,Title,2007-01-04,1.1451,-0.0309,0,-0.026276,nymex announces start date soft commodity futu...
3,3,3,2007,1,5,Title,2007-01-05,1.1506,0.0055,0,0.004803,india largest coffee chain extends pakistan
4,4,4,2007,1,7,Title,2007-01-07,1.1506,-0.0000,0,0.000000,honduran coffee sales ugandan coffee funds soy...


In [9]:
#save a cleaned copy to inspect
justcleandocs.to_csv('cleandata.tsv', sep='\t', encoding='utf-8')
justcleandocs.to_csv('cleandatacomma.tsv', sep=',', encoding='utf-8',header=True)

In [25]:
documents = justcleandocs['Clean_Text']

In [26]:
len(documents)

1458

In [28]:
#spot check
documents[:]


0                  india earns more higher coffee exports
1       friesland raises stake indonesian subsidiary m...
2       nymex announces start date soft commodity futu...
3             india largest coffee chain extends pakistan
4       honduran coffee sales ugandan coffee funds soy...
5                mcdonald coffee be certified sustainable
6       milk product prices europe us milk product pri...
7       fall crude sees sugar hit three month low mcdo...
8                       coke invests australian operation
9       world coffee production fall below demand ico ...
10      milk product prices europe us milk product pri...
11      australian investment nestle scrutinise its in...
12      starbucks denies wwf allegations over coffee p...
13      weather fight acreage sends soyabeans higher h...
14      officials ethiopia starbucks meet geneva ugand...
15      milk product prices europe us monsanto concede...
16      louis dreyfus brazil push selling elevator fir...
17          in

In [29]:
#############################################################################################################################################
#
# Stock future performance classification based on text
#
# Approach:
#
# Build on top of it a 1D convolutional neural network, ending in a softmax output over 3 even categories.
# Use word Glove word vectors for large English text corpus as inputs model
#
# Steps
# 1) After cleaning, we convert all text samples in the dataset into sequences of word indices.  In this case, a "word index" would simply be an integer ID for the word. 
# 2) We consider the top 350,000 most commonly occuring words in the dataset
# 3) We truncate the sequences to a maximum length of 25,000 words.
# 5) We [repare an "embedding matrix" which will contain at index i the embedding vector for the word of index i in our word index.
# 6) Then, we load this embedding matrix into a Keras Embedding layer, set to be frozen (its weights, the embedding vectors, will not be updated during training).
#
###############################################################################################################################################

In [31]:
print('post regex documents', documents.head(10))


post regex documents 0               india earns more higher coffee exports
1    friesland raises stake indonesian subsidiary m...
2    nymex announces start date soft commodity futu...
3          india largest coffee chain extends pakistan
4    honduran coffee sales ugandan coffee funds soy...
5             mcdonald coffee be certified sustainable
6    milk product prices europe us milk product pri...
7    fall crude sees sugar hit three month low mcdo...
8                    coke invests australian operation
9    world coffee production fall below demand ico ...
Name: Clean_Text, dtype: object


In [32]:
MAX_SEQUENCE_LENGTH = 10000
MAX_NB_WORDS = 400000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2
LEARNING_RATE = .00011
BATCH_SIZE = 32
DROPOUT_RATE = 0.45
INNERLAYER_DROPOUT_RATE = 0.15
np.random.seed(2032)

In [35]:
justcleandocs.head()

,Unnamed: 0,Unnamed: 0.1,year,month,day,level_3,Date,Price,Price_Change,Direction,Rate_of_Change,Clean_Text
0,0,0,2007,1,2,Title,2007-01-02,1.1506,0.0000,0,0.000000,india earns more higher coffee exports
1,1,1,2007,1,3,Title,2007-01-03,1.1760,0.0254,0,0.022075,friesland raises stake indonesian subsidiary m...
2,2,2,2007,1,4,Title,2007-01-04,1.1451,-0.0309,0,-0.026276,nymex announces start date soft commodity futu...
3,3,3,2007,1,5,Title,2007-01-05,1.1506,0.0055,0,0.004803,india largest coffee chain extends pakistan
4,4,4,2007,1,7,Title,2007-01-07,1.1506,-0.0000,0,0.000000,honduran coffee sales ugandan coffee funds soy...


In [43]:
justlabels=thedata.drop(['Clean_Text'], axis=1)
justlabels=pd.DataFrame(justlabels[["Price_Change", 'Rate_of_Change']])
justlabels.head()

,Price_Change,Rate_of_Change
0,0.0000,0.000000
1,0.0254,0.022075
2,-0.0309,-0.026276
3,0.0055,0.004803
4,-0.0000,0.000000


In [44]:
######################################################
# Format our text samples and labels for use in Keras
######################################################
# Then we can format our text samples and labels into tensors that can be fed into a neural network. 
# Here we tokenize our source 'justcleandocs'
# note that the values here are ultimately indexes to the actual words

#convert text format
justcleandocslist  = documents.values
justcleandocslist[6]
labels  = justlabels.values
labels_index = {}
#labels_index =  {0:0,1:1,2:2,3:3,4:4}
labels_index =  {0:0,1:1,2:2}
print('labels_index', labels_index)

#tokenize the text
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(justcleandocslist) #tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(justcleandocslist) #sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index #word_index = tokenizer.word_index
print('Found {} unique tokens'.format(len(word_index)))
#print('sequences first', sequences[0])

#Pad sequences so that they all have the same length in a batch of input data 
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', truncating='pre')
sequences = None
texts = None

labels_index {0: 0, 1: 1, 2: 2}
Found 3529 unique tokens


In [45]:
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor: ', data.shape)
print('Shape of label tensor: ', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
X_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('length of y_val',len(y_val))
print('shape of y_val',y_val.shape)
print('length of X_val',len(X_val))
print('shape of X_val',X_val.shape)

Shape of data tensor:  (1458, 10000)
Shape of label tensor:  (1458, 2, 1)
length of y_val 291
shape of y_val (291, 2, 1)
length of X_val 291
shape of X_val (291, 10000)


In [47]:
# #####################################
# # Save Validation Set for Evaluation
# ####################################
# np.savetxt('y_val_3bin.txt', y_val, delimiter=',')
# np.savetxt('X_val_3bin.txt', X_val,  fmt='%s', delimiter=',')
# print('test and training sets saved to disk for later evaluation')

In [48]:
########################################
# Preparing the embedding layer
########################################

#load in word vectors from glove reference global English data set
# https://nlp.stanford.edu/projects/glove/
# see more reference links at bottom

print('Loading word vectors to prepare the embedding layer...')
print(os.getcwd())

embeddings_index = {}
print('Loading Glove Model...')
gloveFile = 'C:\\glove\\glove6B300d.txt'
words = pd.read_table(gloveFile, sep=" ", header=None, quoting=csv.QUOTE_NONE)

print(words.head(5))
print('shape of glove model',words.shape)

wordkeys=words.iloc[:,0]
print('wordkeys type of file', type(wordkeys))
words2 = words.rename(columns={ words.columns[0]: "words" })
words2['words'].apply(str)
#print(words2.dtypes)

embeddings_index = words2.set_index('words').T.to_dict('list')

#print(dict(list(embeddings_index.items())[0:2]))
print('Found {} word vectors.'.format(len(embeddings_index)))
#usage of pandas function dataFrame.to_dict(outtype='dict') outtype : str {‘dict’, ‘list’, ‘series’}


#################################
#Build the embedding matrix
#################################

print('Building Embedding Matrix...')
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Loading word vectors to prepare the embedding layer...
/home/jovyan/Capstone
Loading Glove Model...


NameError: name 'csv' is not defined